In [17]:
import pyfits 
import pandas as pd
from astropy.io import fits
import numpy as np

In [18]:
cannondata=pd.DataFrame()
cannon_param=pyfits.open('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/GALAH_cannon_K2.fits')

In [19]:
#Have to adjust OBJECT to an int to ensure that things are merged correctly with SME data
cannondata['sobject_id']=np.array(cannon_param[1].data['OBJECT']).byteswap().newbyteorder().astype(int)
cannondata['C_GALAH_ID']=np.array(cannon_param[1].data['GALAH_ID']).byteswap().newbyteorder()
cannondata['C_TEFF']=np.array(cannon_param[1].data['TEFF']).byteswap().newbyteorder()
cannondata['C_LOGG']=np.array(cannon_param[1].data['LOGG']).byteswap().newbyteorder()
cannondata['C_FEH']=np.array(cannon_param[1].data['FEH']).byteswap().newbyteorder()
cannondata['C_ALPHA_FE']=np.array(cannon_param[1].data['ALPHA_FE']).byteswap().newbyteorder()
cannondata['EBV']=np.array(cannon_param[1].data['EBV']).byteswap().newbyteorder()
cannondata['C_VEL']=np.array(cannon_param[1].data['VEL']).byteswap().newbyteorder()
cannondata['C_E_VEL']=np.array(cannon_param[1].data['E_VEL']).byteswap().newbyteorder()
cannondata['C_CHI2']=np.array(cannon_param[1].data['CHI2']).byteswap().newbyteorder()

Want to merge with information from IRAF output table. Trying below to make it independent of iraf_output_combination.fits. THIS WILL CHANGE WHEN THE IRAF OUTPUT FILE SYSTEM CHANGES.  

In [20]:
iraf_data=pd.read_csv("/Users/kschles/Documents/GALAH/iraf_v5.1/iraf_dr51_new_05042016.csv")
## The GUESS values are often set to 999. or 9999. rather than null. So I need to adjust this. 
iraf_data.loc[np.where(iraf_data['rv_c2_guess'].astype(float)==999.)[0],'rv_c2_guess']=np.nan

In [21]:
wg4data=pd.merge(iraf_data,cannondata,how='right',on=['sobject_id'])

In [22]:
## Have this because not every column that will be output has information right now. 
wg4data['placeholder']=[np.nan for i in range(len(wg4data))]

In [23]:
wg4data['pipeline']=['iraf_5.1' for i in range(len(wg4data))]
wg4data['cannon_vers']=[1.1 for i in range(len(wg4data))]

In [13]:
wg4data.columns.values

array(['sobject_id', 'galah_id', 'cob_id', 'pivot', 'field_id', 'ra',
       'dec', 'ebv', 'barycentric', 'rv_c1_guess', 'rv_c2_guess',
       'rv_c3_guess', 'rv_c4_guess', 'rv_guess', 'e_rv_guess',
       'teff_guess', 'logg_guess', 'feh_guess', 'flag_guess',
       'snr_c1_guess', 'snr_c2_guess', 'snr_c3_guess', 'snr_c4_guess',
       'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf',
       'snr2_c1_iraf', 'snr2_c2_iraf', 'snr2_c3_iraf', 'snr2_c4_iraf',
       'red_flag', 'red_vers', 'SOBJECT_ID', 'C_GALAH_ID', 'C_TEFF',
       'C_LOGG', 'C_FEH', 'C_ALPHA_FE', 'EBV', 'C_VEL', 'C_E_VEL',
       'C_CHI2', 'placeholder', 'pipeline', 'cannon_vers'], dtype=object)

In [28]:
## Output to FITS file 

## Target Information
col01=fits.Column(name='sobject_id', format='K15', array=wg4data['sobject_id'])
col02=fits.Column(name='galah_id',format='K15', array=wg4data['galah_id'].astype(int))
col03=fits.Column(name='ra',format='F',unit='deg', array=wg4data['ra']) # need to crossmatch with other files
col04=fits.Column(name='dec',format='F',unit='deg', array=wg4data['dec'])
col05=fits.Column(name='a_k',format='E',unit='', array=wg4data['placeholder'])
col06=fits.Column(name='ebv',format='E',unit='mag', array=wg4data['ebv'])
col07=fits.Column(name='red_pipeline', format='A10', array=wg4data['pipeline'])
col08=fits.Column(name='red_flags', format='E', array=wg4data['placeholder'])
## Stellar Parameters from SME (when available)
col09=fits.Column(name='teff_cannon',format='E',unit='K', array=wg4data['C_TEFF'])
col10=fits.Column(name='e_teff_cannon',format='E',unit='K', array=wg4data['placeholder'])
col11=fits.Column(name='logg_cannon',format='E', unit='', array=wg4data['C_LOGG'])
col12=fits.Column(name='e_logg_cannon',format='E', unit='', array=wg4data['placeholder'])
col13=fits.Column(name='feh_cannon',format='E', unit='', array=wg4data['C_FEH'])
col14=fits.Column(name='e_feh_cannon',format='E', unit='', array=wg4data['placeholder'])
col15=fits.Column(name='alpha_fe_cannon',format='E', unit='', array=wg4data['C_ALPHA_FE'])
col16=fits.Column(name='e_alpha_fe_cannon',format='E', unit='', array=wg4data['placeholder'])
col17=fits.Column(name='chi2_cannon', format='E', array=wg4data['C_CHI2'])
col18=fits.Column(name='flag_cannon',format='5A', unit='', array=wg4data['placeholder'])
col19=fits.Column(name='vers_cannon',format='E', array=wg4data['cannon_vers'])
col20=fits.Column(name='rv_cannon',format='E', unit='km/s', array=wg4data['C_VEL'])
col21=fits.Column(name='e_rv_cannon',format='E', unit='km/s', array=wg4data['C_E_VEL'])
col22=fits.Column(name='ebv_cannon',format='E', unit='mag', array=wg4data['EBV'])


In [29]:
cols=[col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,\
      col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,\
      col21,col22]

In [20]:
"""
## Output information for each SME abundance individually
cols_ind=cols

output_col=['_abund_sme', '_e_abund_sme', '_aflag_sme', '_chi_sme', '_sn_sme', \
            '_iter_sme', '_time_sme', '_alog_sme']
input_col=['_ABUND', '_EABUND', '_AFLAG', '_CHI', '_SN', '_ITER', '_TIME', '_ALOG']

for i in range(len(sme_abund[1].data['MODE'][0])):
    columnname=sme_abund[1].data['MODE'][0][i]
    if (columnname!=""): 
        for k in range(len(output_col)):
            output=columnname+output_col[k]
            input=columnname+input_col[k]
            if ((output_col[k]=='_ITER_SME') | (output_col[k]=='_AFLAG_SME') | \
                (output_col[k]=='_ALOG_SME')): 
                cols_ind.append(fits.Column(name=output, format='I', array=wg4data[input]))
            else: 
                cols_ind.append(fits.Column(name=output, format='E', array=wg4data[input]))
"""

In [30]:
## Write out FITS file with abundance information for each feature as its own column
#new_cols = fits.ColDefs(cols_ind)
new_cols = fits.ColDefs(cols)
tbhdu=fits.BinTableHDU.from_columns(new_cols)
tbhdu.writeto('wg4_04292016/sobject_iraf_cannon_1.1_k2.fits')

Want to write out a tuple version of the table informaton as well. First need to change all columns that don't have a match in sme_data from 'NaN' to a tuple of NaN

In [33]:
indexer=np.where(wg4data['SME_ABUND_TUP'].isnull())[0]

In [34]:
null_tuple=[np.nan for i in range(0,99)]

In [35]:
for i in range(len(indexer)):
    k=indexer[i]
    wg4data['SME_ABUND_TUP'].set_value(k,null_tuple)
    wg4data['SME_EABUND_TUP'].set_value(k,null_tuple)
    wg4data['SME_AFLAG_TUP'].set_value(k,null_tuple)
    wg4data['SME_ALOG_TUP'].set_value(k,null_tuple)
    wg4data['SME_TIME_TUP'].set_value(k,null_tuple)
    wg4data['SME_SN_TUP'].set_value(k,null_tuple)
    wg4data['SME_CHI_TUP'].set_value(k,null_tuple)
    wg4data['SME_ITER_TUP'].set_value(k,null_tuple)
    

In [36]:
temp=' '.join(sme_abund[1].data['MODE'][0])
features=[temp for i in range(len(wg4data['I_RA']))]
col40=fits.Column(name='FEATURE_SME',format='594A',unit='',array=features)
col41=fits.Column(name='ABUND_SME',format='99E',unit='',array=tuple(wg4data['SME_ABUND_TUP']))
col42=fits.Column(name='E_ABUND_SME',format='99E',unit='',array=tuple(wg4data['SME_EABUND_TUP']))
col43=fits.Column(name='CHI_SME',format='99E',unit='',array=tuple(wg4data['SME_CHI_TUP']))
col44=fits.Column(name='SN_SME',format='99E',unit='',array=tuple(wg4data['SME_SN_TUP']))
col45=fits.Column(name='AFLAG_SME',format='99E',unit='',array=tuple(wg4data['SME_AFLAG_TUP']))
col46=fits.Column(name='ALOG_SME',format='99E',unit='',array=tuple(wg4data['SME_ALOG_TUP']))
col47=fits.Column(name='TIME_SME',format='99E',unit='seconds',array=tuple(wg4data['SME_TIME_TUP']))
col48=fits.Column(name='ITER_SME',format='99E',unit='',array=tuple(wg4data['SME_ITER_TUP']))


In [37]:
cols=fits.ColDefs([col01,col02,col03,col04,col05,col06,col07,col08,col09,col10,\
                   col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,\
                   col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,\
                   col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,\
                   col41,col42,col43,col44,col45,col46,col47,col48])
tbhdu=fits.BinTableHDU.from_columns(cols)
tbhdu.writeto('wg4_output_tuples.fits')

